In [0]:
#Imports
import cv2
import matplotlib.pyplot as plt
from google.colab import drive
import os
from google.colab import files

In [0]:
def setup_google_drive(forced_remount=False, fresh_install=False):
  print("------------------------------SETTING UP GOOGLE DRIVE------------------------------")
  
  #Force mount Google Drive based on user parameter passed
  #In case of error, restart runtime
  drive.mount('/content/gdrive', force_remount=forced_remount)
  
  #if running for the first time, or the repo is not
  #cloned, or you want to start afresh 
  if fresh_install == True:
    
    !rm -fr /content/gdrive/My\ Drive/darknetProject/darknet
    !git clone https://github.com/AlexeyAB/darknet/ /content/gdrive/My\ Drive/darknetProject/darknet 

In [0]:
def setup_cuda(install_dependencies=False):
  print("------------------------------SETTING UP CUDA------------------------------")
  
  if install_dependencies==True:
    #Downloading and Installing CUDA pre-requisites
    !rm -f /content/gdrive/My\ Drive/darknetProject/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
    !wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb -P /content/gdrive/My\ Drive/darknetProject/
    !dpkg -i /content/gdrive/My\ Drive/darknetProject/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

  !apt update

  #Installing CUDA
  !apt install cuda-8-0 -y

  # If there is an issue installing CUDA, 
  # try installing cuda-visual-tools again
  # Comment the earlier command to install cuda
  # Uncomment and run the below command

  #!apt --fix-broken install cuda-visual-tools-8-0 -y

In [0]:
def setup_compilers():
  print("------------------------------SETTING UP COMPILERS------------------------------")
  
  #Installing compilers
  !apt install gcc-5 g++-5 -y

  #Link compilers to cuda
  !ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
  !ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

In [0]:
def add_to_path():
  print("------------------------------SETTING UP PATH/LIBOPENCV------------------------------")
  
  #Add cuda to PATH
  if not os.environ['PATH'].endswith(':/usr/local/cuda/bin'): 
    os.environ['PATH'] += ':/usr/local/cuda/bin'
    
  # Apparently we need to install this so that OpenCV can work without any issues
  # when we are making the file
  !apt-get install libopencv-dev

In [0]:
def setup_makefile(enable_opencv = True, enable_GPU = True, enable_CUDA = False):
  print("------------------------------SETTING UP MAKEFILE------------------------------")
  
  
  #Changing the variables to include OpenCV and GPU in the Makefile
  if(enable_opencv == True):
    !sed -i 's/OPENCV=0/OPENCV=1/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
  else:
    !sed -i 's/OPENCV=1/OPENCV=0/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
    
  if(enable_GPU == True):
    !sed -i 's/GPU=0/GPU=1/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
  else:
    !sed -i 's/GPU=1/GPU=0/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
  
  if(enable_CUDA == True):
    !sed -i 's/CUDA=0/CUDA=1/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
  else:
    !sed -i 's/CUDA=1/CUDA=0/g' /content/gdrive/My\ Drive/darknetProject/darknet/Makefile
  
  !cat /content/gdrive/My\ Drive/darknetProject/darknet/Makefile

In [0]:
def setup_yolo_weights():
  print("------------------------------SETTING UP NEURAL NETWORK WEIGHTS------------------------------")
  
  import os
  exists = os. path. isfile('/content/gdrive/My\ Drive/darknetProject/darknet/yolov3.weights')
  if not exists:
    !wget https://pjreddie.com/media/files/yolov3.weights -P /content/gdrive/My\ Drive/darknetProject/darknet

In [0]:
def make(enable_opencv = True, enable_GPU = True, enable_CUDA = False):
  
  #Running make!
  print(" - - - - - - - - Makefile - - - - - - - - -")
  setup_makefile(enable_opencv, enable_GPU, enable_CUDA)
  print("------------------------------RUNNING MAKEFILE------------------------------")

  print(" - - - - - - - - Starting Make- - - - - - - - -")  
  !make
  print(" - - - - - - - - Make Complete!- - - - - - - - -")  
  
def setup_environment(forced_remount_drive=False, fresh_install=False, install_CUDA_dependencies=False):
  setup_google_drive(forced_remount_drive, fresh_install)
  setup_cuda(install_CUDA_dependencies)
  setup_compilers()
  add_to_path()
  setup_yolo_weights()

In [0]:
def detect_in_image(path_to_image):
# Here is an example from YOLO
#   !./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg
  !./darknet detect cfg/yolov3.cfg yolov3.weights path_to_image
  %matplotlib inline

  image = cv2.imread("predictions.jpg")
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


In [0]:
def video_predict():
  print("------------------------------PREDICTING ON VIDEO------------------------------")
  
  !./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show /content/gdrive/My\ Drive/darknetProject/darknet/data/input.mp4 -i 0 -out_filename /content/gdrive/My\ Drive/darknetProject/darknet/data/output.mp4
  print(" - - - - - - -- - - - - - Processing Complete! Start Download ! : - - - - - - -- - - - - - ")

def download_processed_video():
    files.download('/content/gdrive/My Drive/darknetProject/darknet/data/output.mp4')

In [0]:
#MAIN FUNCTION for FIRST RUN
#run this only once during start of analysis
def first_run(forced_remount_drive=False, fresh_install=False, install_CUDA_dependencies=False):
  setup_environment(forced_remount_drive, fresh_install, install_CUDA_dependencies)
  %cd /content/gdrive/My\ Drive/darknetProject/darknet/

first_run(True, True, True)

------------------------------SETTING UP GOOGLE DRIVE------------------------------
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into '/content/gdrive/My Drive/darknetProject/darknet'...
remote: Enumerating objects: 9815, done.
remote: Total 9815 (delta 0), reused 0 (delta 0), pack-reused 9815
Receiving objects: 100% (9815/9815), 9.87 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (6557/6557), done.
Checking out files: 100% (1952/1952), done.
------------------------------SETTING UP CUDA--------

In [0]:
#MAIN FUNCTION for OTHER RUNS with MAKE
make(enable_opencv = True, enable_GPU = True, enable_CUDA = False)

 - - - - - - - - Makefile - - - - - - - - -
------------------------------SETTING UP MAKEFILE------------------------------
GPU=1
CUDNN=0
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)

DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	  -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -ge

In [0]:

# ###########################################################################################
# print("------------------------------UPLOAD VIDEO------------------------------")         #  
# uploaded = files.upload()                                                                 #
# for name, data in uploaded.items():                                                       #
#   with open('/content/gdrive/My Drive/darknetProject/data/input.mp4', 'wb') as f:         #
#     f.write(data)                                                                         #
#     print ('saved file', name)                                                            #
# ###########################################################################################


video_predict()

------------------------------PREDICTING ON VIDEO------------------------------
Demo
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF

In [0]:
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show /content/gdrive/My\ Drive/darknetProject/darknet/data/input.mp4 -i 0 -out_filename output.mp4 -ext_output>result.csv

layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  14 c

In [0]:
download_processed_video()

In [0]:
!ls cfg